# Predictions

In [1]:
import sys

sys.path.append('..')

In [2]:
from src.dataset import DatasetGenerator
from src.model import RandomForestPredictor

In [3]:
generator = DatasetGenerator('../data/df.csv', generate_lags=False)

In [4]:
(X_train, y_train), (X_test, y_test) = generator.split_train_test()
predictor = RandomForestPredictor('../checkpoints/rf.pkl')

In [5]:
predictor.train(X_train, y_train)

Model already trained. Skipping...


In [6]:
predictor.eval(X_train, y_train, apply_cv=True)

{'test_neg_root_mean_squared_error': -35.53035933126687,
 'test_neg_mean_absolute_error': -3.6785787036022133,
 'test_explained_variance': 0.9629163138388149}

In [7]:
predictor.eval(X_test, y_test, apply_cv=False)

{'neg_root_mean_squared_error': -31.04314514925384,
 'neg_mean_absolute_error': -3.179801536183558,
 'explained_variance': 0.9710173031043838}